Beat-n game with Nash-env derived from 10 pt limit and augmented with higher-cost sub-Nash Spuds.

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [10]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [11]:
lib10 = get_library(10)
lib10

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [12]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [13]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [15]:
lib=filter_nondominated2(lib10)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

In [16]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [17]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));

In [18]:
s2ind[lib[5]]

5

In [19]:
using JuMP
using HiGHS

In [26]:
vector_model = Model(HiGHS.Optimizer)
n_nash = length(lib)
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv);
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
transpose(b);
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);
@constraint(vector_model, A * x .== b);
@objective(vector_model, Min, c' * x)
@time optimize!(vector_model)

A JuMP Model
Minimization problem with:
Variables: 149
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 75 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 149 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x

In [28]:
w_lp = [value(x[i]) for i in 1:n_nash]
sum(w_lp)

0.9999999999999977

In [29]:
w_lp = w_lp./sum(w_lp);
maximum([sum(payoffs[i, :] .* w_lp) for i in 1:n_nash])

6.38378239159465e-15

In [30]:
new_env = lib[w_lp .> 0]
w_new = w_lp[w_lp .> 0];

In [39]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

for i in sortperm(-w_new)
    print(spud2int(new_env[i]))
    print(" ")
    println(string(w_new[i])[1:6])
end

313111 0.1350
214111 0.1224
131311 0.0835
131221 0.0835
121321 0.0808
113122 0.0543
121312 0.0526
111232 0.0522
121132 0.0506
212122 0.0480
312112 0.0376
213112 0.0371
121123 0.0357
113311 0.0218
111133 0.0180
131122 0.0164
112312 0.0148
122212 0.0142
122311 0.0122
131212 0.0089
112132 0.0077
111412 0.0075
113212 0.0041


In [106]:
function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

function stat_dominates(a::Spud, b::Spud)::Bool
    if minimum(spud_to_vec(a) .- spud_to_vec(b))==0
        return true
    else
        return false
    end
end

function stat_dominates_any(a::Spud, bs::Array{Spud})::Bool
    for b in bs
        if stat_dominates(a, b)
            return true
        end
    end
    return false
end

stat_dominates_any (generic function with 1 method)

In [125]:
libx = get_library(10)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
lib = unique(vcat(new_env, libp))
for ff in lib
    print(spud2int(ff))
    print(", ")
end

111133, 111232, 111412, 112132, 112312, 113122, 113212, 113311, 121123, 121132, 121312, 121321, 122212, 122311, 131122, 131212, 131221, 131311, 212122, 213112, 214111, 312112, 313111, 112123, 112222, 114112, 121222, 121411, 211132, 212131, 213121, 412111, 

In [126]:
libx = get_library(11)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

111341, 124121, 142112, 211322, 241112, 332111, 

In [127]:
libx = get_library(12)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

242121, 243111, 431121, 

In [128]:
libx = get_library(13)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

## Combo analysis

In [129]:
payoffs = get_payoffs(lib)
nlib = length(lib)
libi = map(spud2int, lib);

In [130]:
combos = Array{Int64}(undef, (10000,3))
#ord = sortperm([rand() for i in 1:nlib])
ord = [i for i in 1:nlib]
nc = 0
# 3-combos
for i in ord[1:(nlib-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(nlib-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            print(".")
            for k in ord[(j+1):nlib]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = libi[[i,j,k]]
                    println("")
                    println("----3-COMBO!-----")
                    println(libi[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:]

1 .....
----3-COMBO!-----
[111133, 113122, 332111]
...
----3-COMBO!-----
[111133, 121123, 213112]
..
----3-COMBO!-----
[111133, 121312, 213112]

----3-COMBO!-----
[111133, 121312, 213121]

----3-COMBO!-----
[111133, 121312, 243111]
..
----3-COMBO!-----
[111133, 122212, 213112]

----3-COMBO!-----
[111133, 122212, 214111]

----3-COMBO!-----
[111133, 122212, 213121]

----3-COMBO!-----
[111133, 122212, 243111]
...
----3-COMBO!-----
[111133, 131212, 213112]

----3-COMBO!-----
[111133, 131212, 214111]

----3-COMBO!-----
[111133, 131212, 313111]

----3-COMBO!-----
[111133, 131212, 212131]

----3-COMBO!-----
[111133, 131212, 213121]

----3-COMBO!-----
[111133, 131212, 242121]

----3-COMBO!-----
[111133, 131212, 243111]
.
----3-COMBO!-----
[111133, 131221, 213112]

----3-COMBO!-----
[111133, 131221, 313111]
...
----3-COMBO!-----
[111133, 213112, 114112]

----3-COMBO!-----
[111133, 213112, 121222]

----3-COMBO!-----
[111133, 213112, 124121]
.
----3-COMBO!-----
[111133, 214111, 114112]

----3-COM

211×3 Matrix{Int64}:
 111133  113122  332111
 111133  121123  213112
 111133  121312  213112
 111133  121312  213121
 111133  121312  243111
 111133  122212  213112
 111133  122212  214111
 111133  122212  213121
 111133  122212  243111
 111133  131212  213112
 111133  131212  214111
 111133  131212  313111
 111133  131212  212131
      ⋮          
 313111  211132  124121
 313111  124121  211322
 114112  211132  212131
 114112  211132  213121
 114112  211132  242121
 114112  211132  243111
 114112  212131  211322
 114112  211322  242121
 121222  211132  213121
 121222  211132  243111
 213121  211322  431121
 211322  243111  431121

In [131]:
size(combos)

(211, 3)